# Image Recommendation System via Collaborative Filtering

# ***Please read the instructions very carefully***

1.   Assignment must be implemented in Python 3 only.
2.   You are allowed to use libraries for data preprocessing (numpy, pandas, nltk etc) and for evaluation metrics, data visualization (matplotlib etc.).
3.   You will be evaluated not just on the overall performance of the model and also on the experimentation with hyper parameters, data prepossessing techniques etc.
4.   ⚠️ The Assignment will be evaluated automatically. Please adhere to taking proper inputs from `config.csv` file. You can change your `config.csv` file to experiment with your code. But at the end, make sure that your outputs are corresponding to input values in `config.csv`
5.   Strict plagiarism checking will be done. An F will be awarded for plagiarism.

## About the Dataset
Behance is a community art website where users showcase and discover creative work. Each user is able to “appreciate” (equivalent to a “like” on Instagram or a “react” on Facebook) an image, indicating that they like the image. It is in the website’s best interests to show users pictures that they would like, to keep them engaged for longer. For this question, given a set of pictures that a user has already appreciated, you have to show them a new picture that they would like based on what similar users appreciated.


<br><br>
**The dataset has information of 1 million appreciates of 63,497 users on 178,788 items. The file Behance appreciate 1M has a triplet in each line in the form of (user id, item id, unix timestamp).**

**Task: Take the inputs from the config.csv file and output the recommendations for a particular person**
- Collaborative Filtering is a way to predict items to the user based on the the
user’s history and the history of similar users. The similarity between users can be quantified by the number of images that both the users appreciated.
- The images appreciated by a similar user would be the most suitable images to show a user. Since we can find the similarity between any two users, we would be able to find the “nearest” neighbours of any user, allowing us to use a KNN-based algorithm to recommend new images to a user.
- Since people do not like seeing pictures that they have seen already. Make sure that you do not recommend pictures that a user has appreciated already.
- Output the final response will be saved in the file named ```config['output_file']```.


**Output file format:**
Populate the output file with images that the user has not seen of the k most
similar users, in descending order of their similarity. Each line in the output
file should be a duplet in the form of (item id, user id), where the user id is the
id of the kth similar user. The order of the images corresponding to the same
similar user would not matter. The output file would look something like this:
```
item_id_1_of_1st_similar_user 1st_most_similar_user_id
item_id_2_of_1st_similar_user 1st_most_similar_user_id
item_id_3_of_1st_similar_user 1st_most_similar_user_id
...
item_id_1_of_2nd_similar_user 2nd_most_similar_user_id
item_id_2_of_2nd_similar_user 2nd_most_similar_user_id
item_id_3_of_2nd_similar_user 2nd_most_similar_user_id
...
item_id_1_of_kth_similar_user kth_most_similar_user_id
item_id_2_of_kth_similar_user kth_most_similar_user_id
item_id_3_of_kth_similar_user kth_most_similar_user_id
```

The dataset was extracted using Behance’s API as a part of the paper
“Vista: A visually, socially, and temporally-aware model for artistic
recommendation, RecSys, 2016”. Check out this [Google Drive folder](https://drive.google.com/drive/folders/0B9Ck8jw-TZUEc3NlMjVXdDlPU1k?resourcekey=0-6_8ykn0o4fLc5fuTEm91xA) for
more information about the dataset.


Have fun! The users are waiting to see new pictures!

### Import necessary libraries

In [26]:
# from google.colab import drive
# drive.mount('/content/drive')

In [27]:
# %cd /content/drive/MyDrive

In [28]:
import numpy as np
import pandas as pd

In [29]:
# df=pd.read_csv('Behance_appreciate_1M',sep=' ')

In [30]:
# df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True)

In [31]:
# Config Generation Sample Code.
# ⚠️ Only for experimentation on your side.
# ⚠️ Should be commented during submission.

# df = pd.DataFrame(data=[{'id':276633,
#                   'k':5,
#                   'dataset_file':'./Behance_appreciate_1M',
#                   'output_file':'./output.txt'}])
# df.to_csv('config.csv')

In [32]:
config = pd.read_csv('config.csv').iloc[0]

In [33]:
config

Unnamed: 0                            0
id                               276633
k                                     5
dataset_file    ./Behance_appreciate_1M
output_file                ./output.txt
Name: 0, dtype: object

In [34]:
user = config['id']
k_value = config['k']

### Read the Data

In [35]:
with open(config['dataset_file'], 'r') as inFile:
    appreciate_data = inFile.readlines()

### Initialize a dictionary to store the item_ids that a user likes

### Go through each line of the input file and construct the user_likes dictionary

In [36]:
user_likes = dict()

In [37]:
user_set=set()
image_set=set()

In [38]:
for line in appreciate_data:
    line = line.strip()
    
    user_id = (int(line.split()[0]))
    item_id = (int(line.split()[1]))
    user_set.add(user_id)
    image_set.add(item_id)
    if user_id not in user_likes:
      user_likes[user_id] = list()
    
    user_likes[user_id].append(item_id)

In [39]:
user_dict=dict()
image_dict=dict()
i=0
for id in user_set:
  user_dict[id]=i
  i+=1
i=0
for id in image_set:
  image_dict[id]=i
  i+=1

### Use KNN after Collaborative Filtering to find nearest neighbors

In [40]:
user_set=set(user_set)
image_set=set(image_set)

In [41]:
row=(len(user_set))

In [42]:
col=(len(image_set))

In [43]:
#create sparse matrix
r=[]
c=[]
v=[]
for k in user_likes.keys():
  for j in user_likes[k]:
    r.append(user_dict[k])
    c.append(image_dict[j])
    v.append(1)

In [44]:
r=np.array(r)
c=np.array(c)

In [45]:
from scipy.sparse import csr_matrix
mat = csr_matrix((v, (r, c)),shape = (row, col))

In [46]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=1000, random_state=17)
reduced_matrix=svd.fit_transform(mat)

In [47]:
def cosine_distance(v1,v2):
    "compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    dotp=np.dot(v1,v2)
    mod1=np.sqrt(v1.dot(v1.T))
    mod2=np.sqrt(v2.dot(v2.T))
    return 1.0-float(dotp/(mod1*mod2))

In [48]:

def neighbors(user,k_value):
    """ returns an iterable object (like list or generator) """
    list1=[]
    list2=[]
    dist=list()
    dicto=dict()
    ind=-1
    for val in user_dict.keys():
      if val==user:
        ind=user_dict[val]
    for i in (user_dict.keys()):
      if user_dict[i]!=ind:
        d=float(cosine_distance(reduced_matrix[user_dict[i]],reduced_matrix[ind]))
        dist.append((i,d))
    dist.sort(key=lambda tup: tup[1])
    op=[]
    for i in range(len(dist)):
      op.append(dist[i][0])
    return op
    pass

### Open the output file to write all the lines to the file

In [49]:
print(neighbors(user,k_value))

[1494939, 1973004, 1480429, 2452817, 3660527, 2539765, 148706, 1525134, 3103074, 3924047, 2625832, 3308864, 2492414, 1440929, 417306, 1999429, 3109535, 1459742, 3161839, 1331915, 3270366, 2722917, 3189878, 1262734, 2077107, 2724420, 1305266, 2388318, 1568283, 2103952, 3307481, 1797810, 250586, 839301, 2093263, 1277330, 2731483, 1310452, 2551461, 938743, 1640984, 750657, 1273041, 2679266, 1676949, 241333, 724079, 1388819, 2758717, 2000867, 3107125, 1657315, 2340728, 3929957, 3281034, 3074123, 527817, 843741, 2259611, 1193855, 1444675, 3671943, 328266, 3587384, 2818339, 2126422, 1178717, 141805, 1766688, 1812556, 318158, 589308, 2346155, 2629239, 1287486, 2884639, 2604034, 538817, 2324542, 2175619, 587857, 2099560, 2252152, 1584724, 3921399, 3146180, 1907266, 1815039, 1184248, 3349178, 2303051, 3156398, 1203804, 1261665, 999865, 1458625, 2398492, 435245, 2121893, 1433017, 3666650, 3428094, 3709864, 1991154, 260919, 1534622, 3199594, 2110861, 329214, 689593, 1036949, 173578, 941896, 19669

In [50]:
outFile = open(config['output_file'], 'w')
orignal=set(user_likes[user])
recommended=set()
total=orignal
cnt=0
for n_user in neighbors(user,k_value):
    # user_id = list(user_likes.keys())[n_user]
    if cnt>=k_value:
      break
    recommended=  set(user_likes[n_user])
    toadd=recommended.difference(orignal)
    if len(toadd) >0:
      cnt+=1
    # total=total.union(toadd)
    for item_id in toadd:
        outFile.write(str(item_id) + ' ' + str(n_user) + '\n')

outFile.close()